In [8]:
import pandas as pd
import numpy as np
import datetime
import pickle
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

Predict PMI/ADD using Gelderman's outdoor formulas.

In [14]:
# import Gelderman SOD-labeled data
g_data = pd.read_pickle('./data_max_3months')
print(len(g_data))
display(g_data.head())

176


,TDS,PMI_days,log_PMI_days,age_at_death,sex_male,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,BMI,BMI_status
0,9.0,15.0,2.772589,77.0,0,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.0,0.0,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,30.9,obese
1,13.0,79.0,4.382027,38.0,1,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.0,0.0,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,68.1,obese
3,13.0,50.0,3.931826,73.0,1,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.0,0.0,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,31.0,obese
4,6.0,11.0,2.484907,90.0,0,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.0,0.0,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,25.1,overweight
5,9.0,66.0,4.204693,64.0,0,157.0,60.000000,3.0,3.0,3.0,0,1,0,6.489965,657.500496,616.025357,461.613472,254.476389,41.970833,0.0,0.0,15.961,3.236,66.252,5.476,14.144,3.640,69.846,12.858,10.503,4.683,70.781,15.763,10.276,5.622,75.338,16.508,4.996,6.579,76.685,14.145,21.018,4.598,72.179,11.084,30.7,obese


# Predict PMI

In [15]:
# Apply Gelderman's PMI formula
def pmi(TDS):
    pmi = 10**(-0.93 + (0.18*TDS))
    return pmi

g_data['pred_PMI'] = g_data['TDS'].apply(pmi)
display(g_data.head())

,TDS,PMI_days,log_PMI_days,age_at_death,sex_male,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,BMI,BMI_status,pred_PMI
0,9.0,15.0,2.772589,77.0,0,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.0,0.0,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,30.9,obese,4.897788
1,13.0,79.0,4.382027,38.0,1,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.0,0.0,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,68.1,obese,25.703958
3,13.0,50.0,3.931826,73.0,1,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.0,0.0,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,31.0,obese,25.703958
4,6.0,11.0,2.484907,90.0,0,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.0,0.0,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,25.1,overweight,1.412538
5,9.0,66.0,4.204693,64.0,0,157.0,60.000000,3.0,3.0,3.0,0,1,0,6.489965,657.500496,616.025357,461.613472,254.476389,41.970833,0.0,0.0,15.961,3.236,66.252,5.476,14.144,3.640,69.846,12.858,10.503,4.683,70.781,15.763,10.276,5.622,75.338,16.508,4.996,6.579,76.685,14.145,21.018,4.598,72.179,11.084,30.7,obese,4.897788


# Predict ADD

In [16]:
# Apply Gelderman's ADD formula to the SOD predictions
def add(TDS):
    add = 10**(0.03 + (0.19*TDS))
    return add

g_data['pred_ADD_thres0'] = g_data['TDS'].apply(add)
display(g_data.head())

,TDS,PMI_days,log_PMI_days,age_at_death,sex_male,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,BMI,BMI_status,pred_PMI,pred_ADD_thres0
0,9.0,15.0,2.772589,77.0,0,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.0,0.0,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,30.9,obese,4.897788,54.954087
1,13.0,79.0,4.382027,38.0,1,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.0,0.0,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,68.1,obese,25.703958,316.227766
3,13.0,50.0,3.931826,73.0,1,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.0,0.0,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,31.0,obese,25.703958,316.227766
4,6.0,11.0,2.484907,90.0,0,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.0,0.0,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,25.1,overweight,1.412538,14.791084
5,9.0,66.0,4.204693,64.0,0,157.0,60.000000,3.0,3.0,3.0,0,1,0,6.489965,657.500496,616.025357,461.613472,254.476389,41.970833,0.0,0.0,15.961,3.236,66.252,5.476,14.144,3.640,69.846,12.858,10.503,4.683,70.781,15.763,10.276,5.622,75.338,16.508,4.996,6.579,76.685,14.145,21.018,4.598,72.179,11.084,30.7,obese,4.897788,54.954087


### Derive predicted PMI from predicted ADD
We will now go backwards, meaning deriving the PMI from the predicted ADD. Note, to align with Gelderman's study, a daily temperature below 0 degrees C is counted as 0 since decomposition stops below freezing. Usually, higher temps, faster decomposition, and so smaller PMI, while lower temps, slower decomposition, and so higher PMI. 

In [9]:
# import weather history
weather_df = pd.read_pickle('./../../temp_humidity_data/data/LCD/lcd_daily_avg.pkl')
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d', errors='coerce')
display(weather_df.head())
display(weather_df.info())

,date,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2011-01-01,11.472727,91.803030
1,2011-01-02,3.828571,64.628571
2,2011-01-03,-0.068750,57.812500
3,2011-01-04,3.084375,60.156250
4,2011-01-05,1.008333,83.216667


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4473 entries, 0 to 4472
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      4473 non-null   datetime64[ns]
 1   HourlyDryBulbTemperature  4473 non-null   float64       
 2   HourlyRelativeHumidity    4473 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 139.8 KB


None

In [10]:
# derive PMI from predicted ADD
df_dict = g_data.to_dict('records')
for row in df_dict:
    #print(row)
    discovery_date = row['correct_img_date']
    pred_ADD = row['pred_ADD_thres0']
    
    current_ADD = 0
    derived_PMI = 0
    current_date = discovery_date
    while (current_ADD < pred_ADD):
        temp = weather_df[weather_df.date == current_date]['HourlyDryBulbTemperature'].values[0]
        if temp < 0:
            current_ADD += 0
        else:
            current_ADD += temp
        
        derived_PMI += 1
        
        #print('date:'+str(start_date)+', current_ADD:'+str(current_ADD)+', temp:'+str(temp),  end="\n")
        current_date -= datetime.timedelta(days=1)
        
    row['PMI_from_pred_ADD_thres0'] = derived_PMI
    #print(derived_PMI)
    #break
    #print()

In [11]:
g_data2 = pd.DataFrame.from_dict(df_dict)
display(g_data2.head())
print(g_data2.shape)

,new_id,donor_date,correct_img_date,date_placed_ARF,PMI_days,age_at_death,est_weight_lb,est_stature_in,img_head,true_SOD_G_head,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,sex_male,month,season_of_recovery,fall,spring,summer,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,TDS,pred_PMI,pred_ADD_thres0,PMI_from_pred_ADD_thres0
0,004,00400124,2018-01-24,2018-01-09,15.0,77.0,180.0,64.000000,00400124.12.JPG,3.0,00400124.07.JPG,3.0,00400124.10.JPG,3.0,0,01,winter,0,0,0,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,9.0,4.897788,54.954087,14
1,00b,00b00525,2016-05-25,2016-03-07,79.0,38.0,516.0,73.000039,00b00525.08.JPG,5.0,00b00525.04.JPG,5.0,00b00525.27.JPG,3.0,1,05,spring,0,1,0,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,13.0,25.703958,316.227766,17
2,00b,00b00818,2016-08-18,2016-03-07,164.0,38.0,516.0,73.000039,00b00818.07.JPG,6.0,00b00818.05.JPG,6.0,00b00818.11.JPG,3.0,1,08,summer,0,0,1,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741,15.0,58.884366,758.577575,28
3,00d,00d10116,2019-01-16,2018-11-27,50.0,73.0,235.0,73.000039,00d10116.07.JPG,5.0,00d10116.04.JPG,4.0,00d10116.06.JPG,4.0,1,01,winter,0,0,0,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,13.0,25.703958,316.227766,48
4,011,01101210,2018-12-10,2018-11-29,11.0,90.0,170.0,69.000037,01101210.06.JPG,2.0,01101210.03.JPG,2.0,01101210.08.JPG,2.0,0,12,winter,0,0,0,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,6.0,1.412538,14.791084,5


(256, 55)


# Evaluate results

In [17]:
# Calculate R2 and RMSE (standard error). Since PMI/ADD formulas used were unstranformed (non-logged), data 
# was naturally log transformed for calculating R2.
r2_pmi_logged = r2_score(np.log(g_data.PMI_days+1), np.log(g_data.pred_PMI+1))
rmse_pmi = math.sqrt(mean_squared_error(g_data.PMI_days, g_data.pred_PMI))
rmse_pmi_logged = math.sqrt(mean_squared_error(np.log(g_data.PMI_days+1), np.log(g_data.pred_PMI+1)))
print(r2_pmi_logged, rmse_pmi, rmse_pmi_logged)

-1.4958404243813397 41.56145109528941 1.3919923044404652


In [18]:
# Calculate R2 and RMSE (standard error). Since add/ADD formulas used were unstranformed (non-logged), data 
# was naturally log transformed for calculating R2.
r2_add_logged = r2_score(np.log(g_data.ADD_thres0+1), np.log(g_data.pred_ADD_thres0+1))
rmse_add = math.sqrt(mean_squared_error(g_data.ADD_thres0, g_data.pred_ADD_thres0))
rmse_add_logged = math.sqrt(mean_squared_error(np.log(g_data.ADD_thres0+1), np.log(g_data.pred_ADD_thres0+1)))
print(r2_add_logged, rmse_add, rmse_add_logged)

-1.2442443486490187 511.48055614322493 1.4855039969914174
